# Library Load

In [1]:
# 필요한 PyTorch 라이브러리 불러오기
import os
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.utils import save_image

import PIL
from PIL import Image
import matplotlib.pyplot as plt

import cv2
import numpy as np

import xml.etree.ElementTree as ET

import copy

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [3]:
# GPU 장치 사용 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

# Data Load

In [4]:
# Content Image Path
content_img_folder_path = './rhf_aug_img/'
content_img_folder_list = os.listdir(content_img_folder_path)
content_img_folder_list

['carp', 'cranebird', 'deer', 'lotusflower', 'turtle']

In [5]:
# Style Image Path
style_img_path = './khd_style_img/'
style_img_list = os.listdir(style_img_path)
style_img_list

['b34.jpg', 'b41.jpg', 'b98.jpg']

In [6]:
style_img_name = style_img_list[1]
style_img_name

'b41.jpg'

In [7]:
# 뉴럴 네트워크 모델을 불러옵니다.
cnn = models.vgg19(pretrained=True).features.to(device).eval()
print(cnn)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (17): ReLU(inplace=True)
  (18): MaxPoo

# Function

In [8]:
# 이미지를 불러와 다운받아 텐서(Tensor) 객체로 변환하는 함수
def image_loader(img_path, imsize):
    loader = transforms.Compose([
        transforms.Resize(imsize), # 이미지의 크기를 변경
        transforms.ToTensor() # torch.Tensor 형식으로 변경 [0, 255] → [0, 1]
    ])
    image = PIL.Image.open(img_path)
    # 네트워크 입력에 들어갈 이미지에 배치 목적의 차원(dimension) 추가
    image = loader(image).unsqueeze(0)
    return image.to(device, torch.float) # GPU로 올리기


# torch.Tensor 형태의 이미지를 화면에 출력하는 함수
def imshow(tensor):
    # matplotlib는 CPU 기반이므로 CPU로 옮기기
    image = tensor.cpu().clone()
    # torch.Tensor에서 사용되는 배치 목적의 차원(dimension) 제거
    image = image.squeeze(0)
    # PIL 객체로 변경 
    image = transforms.ToPILImage()(image)
    # 이미지를 화면에 출력(matplotlib는 [0, 1] 사이의 값이라고 해도 정상적으로 처리)
    #plt.imshow(image)
    #plt.show()

In [9]:
# 입력 정규화(Normalization)를 위한 초기화
cnn_normalization_mean = torch.tensor([0.485, 0.456, 0.406]).to(device)
cnn_normalization_std = torch.tensor([0.229, 0.224, 0.225]).to(device)

class Normalization(nn.Module):
    def __init__(self, mean, std):
        super(Normalization, self).__init__()
        self.mean = mean.clone().view(-1, 1, 1)
        self.std = std.clone().view(-1, 1, 1)

    def forward(self, img):
        return (img - self.mean) / self.std

In [10]:
def gram_matrix(input):
    # a는 배치 크기, b는 특징 맵의 개수, (c, d)는 특징 맵의 차원을 의미
    a, b, c, d = input.size()
    # 논문에서는 i = 특징 맵의 개수, j = 각 위치(position)
    features = input.view(a * b, c * d)
    # 행렬 곱으로 한 번에 Gram 내적 계산 가능
    G = torch.mm(features, features.t())
    # Normalize 목적으로 값 나누기
    return G.div(a * b * c * d)


# 스타일 손실(style loss) 계산을 위한 클래스 정의
class StyleLoss(nn.Module):
    def __init__(self, target_feature):
        super(StyleLoss, self).__init__()
        self.target = gram_matrix(target_feature).detach()

    def forward(self, input):
        G = gram_matrix(input)
        self.loss = F.mse_loss(G, self.target)
        return input
    

# 콘텐츠 손실(content loss) 계산을 위한 클래스 정의
class ContentLoss(nn.Module):
    def __init__(self, target,):
        super(ContentLoss, self).__init__()
        self.target = target.detach()

    def forward(self, input):
        self.loss = F.mse_loss(input, self.target)
        return input

In [11]:
content_layers = ['conv_2', 'conv_4']
style_layers = ['conv_1', 'conv_3', 'conv_5']

# Style Transfer 손실(loss)을 계산하는 함수
def get_losses(cnn, content_img, style_img, noise_image):
    cnn = copy.deepcopy(cnn)
    normalization = Normalization(cnn_normalization_mean, cnn_normalization_std).to(device)
    content_losses = []
    style_losses = []
    
    # 가장 먼저 입력 이미지가 입력 정규화(input normalization)를 수행하도록
    model = nn.Sequential(normalization)

    # 현재 CNN 모델에 포함되어 있는 모든 레이어를 확인하며
    i = 0
    for layer in cnn.children():
        if isinstance(layer, nn.Conv2d):
            i += 1
            name = 'conv_{}'.format(i)
        elif isinstance(layer, nn.ReLU):
            name = 'relu_{}'.format(i)
            layer = nn.ReLU(inplace=False)
        elif isinstance(layer, nn.MaxPool2d):
            name = 'pool_{}'.format(i)
        elif isinstance(layer, nn.BatchNorm2d):
            name = 'bn_{}'.format(i)
        else:
            raise RuntimeError('Unrecognized layer: {}'.format(layer.__class__.__name__))

        model.add_module(name, layer)

        # 설정한 content layer까지의 결과를 이용해 content loss를 계산
        if name in content_layers:
            target_feature = model(content_img).detach()
            content_loss = ContentLoss(target_feature)
            model.add_module("content_loss_{}".format(i), content_loss)
            content_losses.append(content_loss)

        # 설정한 style layer까지의 결과를 이용해 style loss를 계산
        if name in style_layers:
            target_feature = model(style_img).detach()
            style_loss = StyleLoss(target_feature)
            model.add_module("style_loss_{}".format(i), style_loss)
            style_losses.append(style_loss)

    # 마지막 loss 이후의 레이어는 사용하지 않도록
    for i in range(len(model) - 1, -1, -1):
        if isinstance(model[i], ContentLoss) or isinstance(model[i], StyleLoss):
            break

    model = model[:(i + 1)]
    return model, content_losses, style_losses

In [12]:
def style_transfer(cnn, content_img, style_img, input_img, iters):
    model, content_losses, style_losses = get_losses(cnn, content_img, style_img, input_img)
    optimizer = optim.LBFGS([input_img.requires_grad_()])

    print("[ Start ]")
    imshow(input_img)

    # 하나의 값만 이용하기 위해 배열 형태로 사용
    run = [0]
    while run[0] <= iters:

        def closure():
            input_img.data.clamp_(0, 1)

            optimizer.zero_grad()
            model(input_img)
            content_score = 0
            style_score = 0

            for cl in content_losses:
                content_score += cl.loss
            for sl in style_losses:
                style_score += sl.loss

            style_score *= 1e5
            loss = content_score + style_score
            loss.backward()

            run[0] += 1
            if run[0] % 100 == 0:
                print(f"[ Step: {run[0]} / Content loss: {content_score.item()} / Style loss: {style_score.item()}]")
                imshow(input_img)
            
            return content_score + style_score
        
        optimizer.step(closure)

    # 결과적으로 이미지의 각 픽셀의 값이 [0, 1] 사이의 값이 되도록 자르기
    input_img.data.clamp_(0, 1)

    return input_img

# Style Transfer

In [13]:
for content_img_folder_name in content_img_folder_list:
    print(content_img_folder_name)
    # 폴더 내의 파일 리스트
    content_img_list = os.listdir(content_img_folder_path + content_img_folder_name)

    #imshow(style_img)
    for content_img_name in content_img_list:
        content_img_size = PIL.Image.open(content_img_folder_path + content_img_folder_name + '/' + content_img_name).size
        content_img = image_loader(content_img_folder_path + content_img_folder_name + '/' + content_img_name, (content_img_size[1], content_img_size[0]))
        #imshow(content_img)

        input_img = torch.empty_like(content_img).uniform_(0, 1).to(device)
        #imshow(input_img)

        style_img = image_loader(style_img_path + style_img_name, (content_img_size[1], content_img_size[0]))

        # style transfer 수행
        output = style_transfer(cnn, content_img=content_img, style_img=style_img, input_img=input_img, iters=1500)

        # 이미지 저장
        save_image(output.cpu().detach()[0], './style_transfer_img/' + style_img_name.split('.')[0] + '/' + content_img_folder_name + '/' + content_img_name.split('.')[0] + '_style1' + '.jpg')
        print('='*60)

carp
[ Start ]
[ Step: 100 / Content loss: 6.684068202972412 / Style loss: 1.9188917875289917]
[ Step: 200 / Content loss: 5.279500961303711 / Style loss: 0.912259042263031]
[ Step: 300 / Content loss: 5.112911224365234 / Style loss: 0.720728874206543]
[ Step: 400 / Content loss: 5.063504219055176 / Style loss: 0.6751230359077454]
[ Step: 500 / Content loss: 5.038783073425293 / Style loss: 0.6603103280067444]
[ Step: 600 / Content loss: 5.021575927734375 / Style loss: 0.6539363265037537]
[ Step: 700 / Content loss: 5.0077362060546875 / Style loss: 0.6504284143447876]
[ Step: 800 / Content loss: 4.994939804077148 / Style loss: 0.6489031910896301]
[ Step: 900 / Content loss: 4.984067916870117 / Style loss: 0.6475759148597717]
[ Step: 1000 / Content loss: 4.973275184631348 / Style loss: 0.6468098759651184]
[ Step: 1100 / Content loss: 4.963492393493652 / Style loss: 0.6458225250244141]
[ Step: 1200 / Content loss: 4.954843521118164 / Style loss: 0.64545738697052]
[ Step: 1300 / Content lo

[ Step: 800 / Content loss: 16.870773315429688 / Style loss: 1.3949295282363892]
[ Step: 900 / Content loss: 16.846590042114258 / Style loss: 1.3970725536346436]
[ Step: 1000 / Content loss: 16.824172973632812 / Style loss: 1.3989360332489014]
[ Step: 1100 / Content loss: 16.804046630859375 / Style loss: 1.4003020524978638]
[ Step: 1200 / Content loss: 16.782764434814453 / Style loss: 1.4040553569793701]
[ Step: 1300 / Content loss: 16.763477325439453 / Style loss: 1.4064995050430298]
[ Step: 1400 / Content loss: 16.748523712158203 / Style loss: 1.4063903093338013]
[ Step: 1500 / Content loss: 16.73336410522461 / Style loss: 1.4076788425445557]
[ Start ]
[ Step: 100 / Content loss: 10.42577838897705 / Style loss: 2.27119517326355]
[ Step: 200 / Content loss: 8.936897277832031 / Style loss: 1.3590059280395508]
[ Step: 300 / Content loss: 8.65081787109375 / Style loss: 1.145041584968567]
[ Step: 400 / Content loss: 8.537128448486328 / Style loss: 1.112375020980835]
[ Step: 500 / Content 

[ Step: 1500 / Content loss: 23.226547241210938 / Style loss: 2.257920742034912]
[ Start ]
[ Step: 100 / Content loss: 9.766048431396484 / Style loss: 2.2777626514434814]
[ Step: 200 / Content loss: 8.325284957885742 / Style loss: 1.3371795415878296]
[ Step: 300 / Content loss: 8.099919319152832 / Style loss: 1.1453748941421509]
[ Step: 400 / Content loss: 8.030744552612305 / Style loss: 1.1114070415496826]
[ Step: 500 / Content loss: 7.995358467102051 / Style loss: 1.1030058860778809]
[ Step: 600 / Content loss: 7.971956253051758 / Style loss: 1.0984352827072144]
[ Step: 700 / Content loss: 7.953286170959473 / Style loss: 1.0953317880630493]
[ Step: 800 / Content loss: 7.935927391052246 / Style loss: 1.0936214923858643]
[ Step: 900 / Content loss: 7.918970108032227 / Style loss: 1.0935406684875488]
[ Step: 1000 / Content loss: 7.903115272521973 / Style loss: 1.0935566425323486]
[ Step: 1100 / Content loss: 7.888180732727051 / Style loss: 1.094689965248108]
[ Step: 1200 / Content loss:

[ Step: 700 / Content loss: 6.594851970672607 / Style loss: 0.6885573267936707]
[ Step: 800 / Content loss: 6.572507381439209 / Style loss: 0.6903555393218994]
[ Step: 900 / Content loss: 6.553287982940674 / Style loss: 0.6911640167236328]
[ Step: 1000 / Content loss: 6.538137435913086 / Style loss: 0.6900100111961365]
[ Step: 1100 / Content loss: 6.5248847007751465 / Style loss: 0.6891190409660339]
[ Step: 1200 / Content loss: 6.514838218688965 / Style loss: 0.6869863867759705]
[ Step: 1300 / Content loss: 6.506287574768066 / Style loss: 0.6852181553840637]
[ Step: 1400 / Content loss: 6.499790191650391 / Style loss: 0.6835960745811462]
[ Step: 1500 / Content loss: 6.494302749633789 / Style loss: 0.6821801066398621]
[ Start ]
[ Step: 100 / Content loss: 22.960447311401367 / Style loss: 3.7231643199920654]
[ Step: 200 / Content loss: 21.386585235595703 / Style loss: 2.948948621749878]
[ Step: 300 / Content loss: 21.12614631652832 / Style loss: 2.793227434158325]
[ Step: 400 / Content l

[ Step: 1400 / Content loss: 2.1717820167541504 / Style loss: 0.6426435708999634]
[ Step: 1500 / Content loss: 2.168999433517456 / Style loss: 0.6423861384391785]
[ Start ]
[ Step: 100 / Content loss: 7.224858283996582 / Style loss: 1.9968607425689697]
[ Step: 200 / Content loss: 5.6439595222473145 / Style loss: 1.0241321325302124]
[ Step: 300 / Content loss: 5.242763519287109 / Style loss: 0.8858385682106018]
[ Step: 400 / Content loss: 5.085127830505371 / Style loss: 0.9043523073196411]
[ Step: 500 / Content loss: 4.998598098754883 / Style loss: 0.9246386289596558]
[ Step: 600 / Content loss: 4.944624423980713 / Style loss: 0.937309205532074]
[ Step: 700 / Content loss: 4.90887451171875 / Style loss: 0.9439892768859863]
[ Step: 800 / Content loss: 4.888418197631836 / Style loss: 0.9446325898170471]
[ Step: 900 / Content loss: 4.8737030029296875 / Style loss: 0.9443579912185669]
[ Step: 1000 / Content loss: 4.863285064697266 / Style loss: 0.9427351951599121]
[ Step: 1100 / Content los

...

[ Step: 900 / Content loss: 3.0244407653808594 / Style loss: 0.5768544673919678]
[ Step: 1000 / Content loss: 3.0131707191467285 / Style loss: 0.5775375962257385]
[ Step: 1100 / Content loss: 3.004047155380249 / Style loss: 0.5785161852836609]
[ Step: 1200 / Content loss: 2.996591329574585 / Style loss: 0.5795491933822632]
[ Step: 1300 / Content loss: 2.990003824234009 / Style loss: 0.5805774927139282]
[ Step: 1400 / Content loss: 2.9845972061157227 / Style loss: 0.5815097093582153]
[ Step: 1500 / Content loss: 2.9795901775360107 / Style loss: 0.5826926827430725]
turtle
[ Start ]
[ Step: 100 / Content loss: 10.01290512084961 / Style loss: 2.320651054382324]
[ Step: 200 / Content loss: 8.147122383117676 / Style loss: 1.4446126222610474]
[ Step: 300 / Content loss: 7.854813575744629 / Style loss: 1.2800791263580322]
[ Step: 400 / Content loss: 7.757211685180664 / Style loss: 1.2253999710083008]
[ Step: 500 / Content loss: 7.71128511428833 / Style loss: 1.201595425605774]
[ Step: 600 / Co

[ Start ]
[ Step: 100 / Content loss: 19.048473358154297 / Style loss: 2.6185498237609863]
[ Step: 200 / Content loss: 17.313087463378906 / Style loss: 1.7017810344696045]
[ Step: 300 / Content loss: 17.003616333007812 / Style loss: 1.5304830074310303]
[ Step: 400 / Content loss: 16.8840389251709 / Style loss: 1.4610353708267212]
[ Step: 500 / Content loss: 16.81172752380371 / Style loss: 1.4395172595977783]
[ Step: 600 / Content loss: 16.765033721923828 / Style loss: 1.4294408559799194]
[ Step: 700 / Content loss: 16.73067855834961 / Style loss: 1.4247828722000122]
[ Step: 800 / Content loss: 16.703773498535156 / Style loss: 1.4222629070281982]
[ Step: 900 / Content loss: 16.680944442749023 / Style loss: 1.42079758644104]
[ Step: 1000 / Content loss: 16.660491943359375 / Style loss: 1.4202913045883179]
[ Step: 1100 / Content loss: 16.641643524169922 / Style loss: 1.4203662872314453]
[ Step: 1200 / Content loss: 16.624465942382812 / Style loss: 1.4201253652572632]
[ Step: 1300 / Conten

[ Step: 700 / Content loss: 4.247007369995117 / Style loss: 0.8274558186531067]
[ Step: 800 / Content loss: 273.1735534667969 / Style loss: 483490.125]
[ Step: 900 / Content loss: 6.100096225738525 / Style loss: 1.844831109046936]
[ Step: 1000 / Content loss: 4.6536383628845215 / Style loss: 1.9426592588424683]
[ Step: 1100 / Content loss: 4.521109580993652 / Style loss: 1.2969828844070435]
[ Step: 1200 / Content loss: 4.423146724700928 / Style loss: 0.8217587471008301]
[ Step: 1300 / Content loss: 392.014892578125 / Style loss: 1015583.8125]
[ Step: 1400 / Content loss: 8.813196182250977 / Style loss: 3.036360740661621]
[ Step: 1500 / Content loss: 4.947439670562744 / Style loss: 1.3313010931015015]
[ Start ]
[ Step: 100 / Content loss: 4.5343098640441895 / Style loss: 1.6563513278961182]
[ Step: 200 / Content loss: 3.028010368347168 / Style loss: 0.8020216226577759]
[ Step: 300 / Content loss: 2.7629904747009277 / Style loss: 0.5899882912635803]
[ Step: 400 / Content loss: 2.68831610

[ Step: 1400 / Content loss: 27.073814392089844 / Style loss: 2.313610076904297]
[ Step: 1500 / Content loss: 27.06666374206543 / Style loss: 2.3058247566223145]
[ Start ]
[ Step: 100 / Content loss: 21.718177795410156 / Style loss: 3.2178781032562256]
[ Step: 200 / Content loss: 20.364437103271484 / Style loss: 2.389458656311035]
[ Step: 300 / Content loss: 20.042715072631836 / Style loss: 2.2356948852539062]
[ Step: 400 / Content loss: 19.894208908081055 / Style loss: 2.1576242446899414]
[ Step: 500 / Content loss: 19.815704345703125 / Style loss: 2.100048303604126]
[ Step: 600 / Content loss: 19.7614688873291 / Style loss: 2.064502477645874]
[ Step: 700 / Content loss: 19.72292709350586 / Style loss: 2.043503761291504]
[ Step: 800 / Content loss: 19.694311141967773 / Style loss: 2.029705762863159]
[ Step: 900 / Content loss: 19.671937942504883 / Style loss: 2.019648790359497]
[ Step: 1000 / Content loss: 19.653690338134766 / Style loss: 2.0133566856384277]
[ Step: 1100 / Content los

[ Step: 500 / Content loss: 10.602980613708496 / Style loss: 1.3764408826828003]
[ Step: 600 / Content loss: 10.559122085571289 / Style loss: 1.370111346244812]
[ Step: 700 / Content loss: 10.523950576782227 / Style loss: 1.3683828115463257]
[ Step: 800 / Content loss: 10.493247985839844 / Style loss: 1.3689841032028198]
[ Step: 900 / Content loss: 10.466726303100586 / Style loss: 1.3694576025009155]
[ Step: 1000 / Content loss: 10.441555976867676 / Style loss: 1.3713431358337402]
[ Step: 1100 / Content loss: 10.417189598083496 / Style loss: 1.3739261627197266]
[ Step: 1200 / Content loss: 10.396090507507324 / Style loss: 1.3759409189224243]
[ Step: 1300 / Content loss: 10.376443862915039 / Style loss: 1.3781497478485107]
[ Step: 1400 / Content loss: 10.358366012573242 / Style loss: 1.3809245824813843]
[ Step: 1500 / Content loss: 10.342368125915527 / Style loss: 1.3836510181427002]
[ Start ]
[ Step: 100 / Content loss: 28.56764030456543 / Style loss: 3.387101650238037]
[ Step: 200 / C